<a href="https://colab.research.google.com/github/parthoBhoumk/cnn_pytorch/blob/main/oldPaperModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
import os 
from os import listdir
from tqdm import tqdm
import shutil

!pip install opendatasets --upgrade --quiet
import opendatasets as od
import random

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix

%matplotlib inline


In [2]:
from tensorflow.keras.models import Model

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [5]:
dataset_url = 'https://www.kaggle.com/ambarish/breakhis'
od.download(dataset_url)

# username: asmiyahasan
# key: f3f15bebbd529ff7bb18c499d50b1c5a

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: asmiyahasan
Your Kaggle Key: ··········


100%|██████████| 3.99G/3.99G [01:27<00:00, 48.8MB/s]


In [6]:
##creating folders to store the resampled images of 8 class

classes = ['A', 'F', 'PT', 'TA', 'DC', 'LC', 'MC', 'PC']

os.mkdir('train')

for item in classes:
  os.mkdir('./train/'+item)

In [7]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [8]:
subclass_names = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma', 
                  'ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

for count, item in enumerate(subclass_names):
  if count < 4:
    file_names =  getListOfFiles('./breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/'+item)
  else:
    file_names =  getListOfFiles('./breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+item)
   
  for f in file_names:
    if f.endswith('.png'):
      shutil.move(f,'train/'+classes[count])

In [9]:
os.mkdir('valid')

for item in classes:
  os.mkdir('./valid/'+item)

In [10]:
for item in classes:
  source = './train/'+item
  dest = './valid/'+item
  files = os.listdir(source)
  no_of_files = len(files) // 10

  for file_name in random.sample(files, no_of_files):
      shutil.move(os.path.join(source, file_name), dest)

In [11]:
F_files = os.listdir('./valid/F')
len(F_files)

len(os.listdir('./'))

5

In [12]:
print('Training Set')
for item in classes:
  print(item,': ', len(os.listdir('train/'+item)))

print('Validation Set')
for item in classes:
  print(item,': ', len(os.listdir('valid/'+item)))

Training Set
A :  400
F :  913
PT :  408
TA :  513
DC :  3106
LC :  564
MC :  713
PC :  504
Validation Set
A :  44
F :  101
PT :  45
TA :  56
DC :  345
LC :  62
MC :  79
PC :  56


In [13]:
train_path = './train'
valid_path = './valid'
#test_path = '' 

In [14]:
aug_size_train = 1800
aug_size_valid = 200

In [15]:
## Resampling training images

for item in classes:
  source = './train/'+item
  files = os.listdir(source)
  additional_size = aug_size_train-len(files)

  if additional_size > 0:
    while len(os.listdir(source)) != aug_size_train:
      file1 = random.choice(os.listdir(source))
      shutil.copyfile(os.path.join(source, file1), os.path.join(source, 'copy-'+file1))
  elif additional_size < 0:
    for file2 in random.sample(files, -additional_size):
    #while len(os.listdir(source)) != aug_size_train:
    #  file2 = random.choice(files)
      os.remove('./train/'+item+'/'+file2)
  else:
    continue

  print(item, ':', len(os.listdir(source)))

A : 1800
F : 1800
PT : 1800
TA : 1800
DC : 1800
LC : 1800
MC : 1800
PC : 1800


In [16]:
## Resampling validation images

for item in classes:
  source = './valid/'+item
  files = os.listdir(source)
  additional_size = aug_size_valid-len(files)

  if additional_size > 0:
    while len(os.listdir(source)) != aug_size_valid:
      file1 = random.choice(os.listdir(source))
      shutil.copyfile(os.path.join(source, file1), os.path.join(source, 'copy-'+file1))
  elif additional_size < 0:
    for file2 in random.sample(files, -additional_size):
    #while len(os.listdir(source)) != aug_size_train:
    #  file2 = random.choice(files)
      os.remove('./valid/'+item+'/'+file2)
  else:
    continue

  print(item, ':', len(os.listdir(source)))

A : 200
F : 200
PT : 200
TA : 200
DC : 200
LC : 200
MC : 200
PC : 200


In [17]:
print('Training Set')
for item in classes:
  print(item,': ', len(os.listdir('train/'+item)))

Training Set
A :  1800
F :  1800
PT :  1800
TA :  1800
DC :  1800
LC :  1800
MC :  1800
PC :  1800


In [18]:
print('validation Set')
for item in classes:
  print(item,': ', len(os.listdir('valid/'+item)))

validation Set
A :  200
F :  200
PT :  200
TA :  200
DC :  200
LC :  200
MC :  200
PC :  200


In [19]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
  #  featurewise_center=True,
   # featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(
    rescale = 1/255,
    ##featurewise_std_normalization=True
    )

In [26]:
img_size = 64

In [27]:
train_batches = train_datagen.flow_from_directory(directory=train_path, target_size=(img_size,img_size),
                                                  class_mode = 'categorical', batch_size=32)

valid_batches = valid_datagen.flow_from_directory(directory=valid_path, target_size=(img_size,img_size),
                                                  class_mode = 'categorical', batch_size=8, shuffle = False)

Found 14400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


In [28]:
k_size = 3
model = Sequential([
    Conv2D(filters=32, kernel_size=(k_size, k_size), activation='relu', padding = 'same', input_shape=(64,64,3)),
    BatchNormalization(),
    MaxPool2D(pool_size=(3, 3), strides=3),

    Conv2D(filters=64, kernel_size=(k_size, k_size), activation='relu', padding = 'same'),
    BatchNormalization(),

    Conv2D(filters=128, kernel_size=(k_size, k_size), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Dropout(0.3),

    Conv2D(filters=256, kernel_size=(k_size, k_size), activation='relu', padding = 'same'),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=(k_size, k_size), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Dropout(0.3),

    Flatten(),
    Dense(units=256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(units=8, activation='softmax')
])

In [ ]:
print(model.summary())

In [29]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x = train_batches,steps_per_epoch=len(train_batches), validation_data=valid_batches,
         validation_steps=len(valid_batches),epochs = 50,verbose = 2) 

Epoch 1/50
450/450 - 330s - loss: 1.5724 - accuracy: 0.4314 - val_loss: 2.4917 - val_accuracy: 0.3044 - 330s/epoch - 733ms/step
Epoch 2/50
450/450 - 307s - loss: 1.1303 - accuracy: 0.5799 - val_loss: 1.8067 - val_accuracy: 0.3963 - 307s/epoch - 682ms/step
Epoch 3/50
450/450 - 299s - loss: 0.9612 - accuracy: 0.6410 - val_loss: 3.2876 - val_accuracy: 0.2900 - 299s/epoch - 665ms/step
Epoch 4/50
450/450 - 299s - loss: 0.8562 - accuracy: 0.6776 - val_loss: 7.2511 - val_accuracy: 0.1469 - 299s/epoch - 664ms/step
Epoch 5/50
450/450 - 298s - loss: 0.7663 - accuracy: 0.7156 - val_loss: 7.9702 - val_accuracy: 0.1475 - 298s/epoch - 663ms/step
Epoch 6/50
450/450 - 303s - loss: 0.7011 - accuracy: 0.7371 - val_loss: 4.0123 - val_accuracy: 0.2506 - 303s/epoch - 673ms/step
Epoch 7/50
450/450 - 302s - loss: 0.6404 - accuracy: 0.7620 - val_loss: 1.8631 - val_accuracy: 0.4644 - 302s/epoch - 672ms/step
Epoch 8/50
450/450 - 298s - loss: 0.5790 - accuracy: 0.7852 - val_loss: 4.0340 - val_accuracy: 0.3137 - 